### This is a auxillary script to generate the bayesian network as *network.json* file, required by libpgm

In [105]:
import json
import xml.etree.cElementTree

In [106]:
root = xml.etree.cElementTree.parse('walrus.xdsl').getroot()

nodes = root.findall('nodes')

In [107]:
n = {}
names = []
edges = []
count = 0

In [108]:
for node in nodes[0].findall('cpt'):
    nodeName = node.get('id')
    vals = []
    probs = []
    parents = None
    children = []
    if node.findall('state'):
        vals = [x.get('id') for x in node.findall('state')]
    probs = [float(x) for x in node.find('probabilities').text.split()]
    if node.find('parents') is not None:
        parents = node.find('parents').text.split()
    n[nodeName] = {
            'ord':count,
            'numoutcomes':len(vals),
            'vals':vals,
            'parents':parents,
            'cprob':probs,
            'children':children
        }
    count = count + 1
    
    #print node.get('id')

In [109]:
import numpy as np
import itertools

m=0

for k,v in n.items():
    result = {}
    if not v['parents']:
        continue
    cpd = np.array(v['cprob'])
    cpd = cpd.reshape(len(cpd)/v['numoutcomes'], v['numoutcomes'])
    numParents = len(v['parents'])
    if numParents == 1:
        parentOne = v['parents'][0]
        parentOneVals = n[parentOne]['vals']
        if len(parentOneVals) != cpd.shape[0]:
            print "Problem!!"
            print k+" = ",v
        for i,pv in enumerate(parentOneVals):
            key = "['"+pv+"']"
            result[key]=cpd[i,:].tolist()
    elif numParents == 2:
        parentOne = v['parents'][0]
        parentOneVals = n[parentOne]['vals']
        parentTwo = v['parents'][1]
        parentTwoVals = n[parentTwo]['vals']
        if len(parentOneVals) * len(parentTwoVals) != cpd.shape[0]:
            print "Problem!!"
            print k+" = ",v
        for i,pv in enumerate(list(itertools.product(parentOneVals,parentTwoVals))):
            key = "['"+pv[0]+"', '"+pv[1]+"']"
            result[key]=cpd[i,:].tolist()
    elif numParents == 3:
        parentOne = v['parents'][0]
        parentOneVals = n[parentOne]['vals']
        parentTwo = v['parents'][1]
        parentTwoVals = n[parentTwo]['vals']
        parentThree = v['parents'][2]
        parentThreeVals = n[parentThree]['vals']
        if len(parentOneVals) * len(parentTwoVals) * len(parentThreeVals) != cpd.shape[0]:
            print "Problem!!"
            print k+" = ",v
        for i,pv in enumerate(list(itertools.product(parentOneVals,parentTwoVals, parentThreeVals))):
            key = "['"+pv[0]+"', '"+pv[1]+"', '"+pv[2]+"']"
            result[key]=cpd[i,:].tolist()
    v['cprob'] = result

In [110]:
for k,v in n.items():
    if not v['parents']:
        continue
    for parentName in v['parents']:
        n[parentName]['children'].append(k)

In [111]:
for k,v in n.items():
    names.append(k)
    if not v['children'] or len(v['children']) < 1:
        continue
    for child in v['children']:
        edges.append([k,child])

In [112]:
edges

[['BENsumfal', 'EEsumfal'],
 ['BENwin', 'EEwin'],
 ['IceBwin', 'SIEwin'],
 ['DMspr', 'HDspr'],
 ['DMspr', 'TMspr'],
 ['RUspr', 'BENspr'],
 ['TMsumfal', 'ASsumfal'],
 ['DPsumfal', 'BODsumfal'],
 ['IceBsumfal', 'SIEsumfal'],
 ['HDwin', 'CDwin'],
 ['AVAsumfal', 'DPsumfal'],
 ['AVAsumfal', 'CRsumfal'],
 ['AVAsumfal', 'EEsumfal'],
 ['AVAsumfal', 'PRsumfal'],
 ['DMwin', 'HDwin'],
 ['DMwin', 'TMwin'],
 ['CDwin', 'TMwin'],
 ['D', 'ASwin'],
 ['IceBspr', 'SIEspr'],
 ['CRsumfal', 'CDsumfal'],
 ['IceMsumfal', 'AVAsumfal'],
 ['OILwin', 'BENwin'],
 ['OILwin', 'BODwin'],
 ['IceMspr', 'AVAspr'],
 ['BENspr', 'EEspr'],
 ['CRwin', 'CDwin'],
 ['IceMwin', 'AVAwin'],
 ['EEsumfal', 'BODsumfal'],
 ['TAOsumfal', 'DMsumfal'],
 ['SIEsumfal', 'AVAsumfal'],
 ['SIEsumfal', 'SIEall'],
 ['CCBsumfal', 'BENsumfal'],
 ['PRwin', 'TMwin'],
 ['CCBwin', 'BENwin'],
 ['IceCspr', 'SIEspr'],
 ['AVAwin', 'D'],
 ['AVAwin', 'CRwin'],
 ['AVAwin', 'PRwin'],
 ['AVAwin', 'EEwin'],
 ['AVAwin', 'DPwin'],
 ['SATsumfal', 'HDsumfal'],
 ['S

In [113]:
output = {}
output['V'] = names
output['E'] = edges
output['Vdata'] = n

In [114]:
output

{'E': [['BENsumfal', 'EEsumfal'],
  ['BENwin', 'EEwin'],
  ['IceBwin', 'SIEwin'],
  ['DMspr', 'HDspr'],
  ['DMspr', 'TMspr'],
  ['RUspr', 'BENspr'],
  ['TMsumfal', 'ASsumfal'],
  ['DPsumfal', 'BODsumfal'],
  ['IceBsumfal', 'SIEsumfal'],
  ['HDwin', 'CDwin'],
  ['AVAsumfal', 'DPsumfal'],
  ['AVAsumfal', 'CRsumfal'],
  ['AVAsumfal', 'EEsumfal'],
  ['AVAsumfal', 'PRsumfal'],
  ['DMwin', 'HDwin'],
  ['DMwin', 'TMwin'],
  ['CDwin', 'TMwin'],
  ['D', 'ASwin'],
  ['IceBspr', 'SIEspr'],
  ['CRsumfal', 'CDsumfal'],
  ['IceMsumfal', 'AVAsumfal'],
  ['OILwin', 'BENwin'],
  ['OILwin', 'BODwin'],
  ['IceMspr', 'AVAspr'],
  ['BENspr', 'EEspr'],
  ['CRwin', 'CDwin'],
  ['IceMwin', 'AVAwin'],
  ['EEsumfal', 'BODsumfal'],
  ['TAOsumfal', 'DMsumfal'],
  ['SIEsumfal', 'AVAsumfal'],
  ['SIEsumfal', 'SIEall'],
  ['CCBsumfal', 'BENsumfal'],
  ['PRwin', 'TMwin'],
  ['CCBwin', 'BENwin'],
  ['IceCspr', 'SIEspr'],
  ['AVAwin', 'D'],
  ['AVAwin', 'CRwin'],
  ['AVAwin', 'PRwin'],
  ['AVAwin', 'EEwin'],
  ['AVAwin

In [115]:
f = open('network.json', 'w+')
f.write(json.dumps(output))
f.close()